# Part 1
## Parsing

In [71]:
import re
pattern = re.compile(r"Valve (\w+) has flow rate=(\d+); tunnels? leads? to valves? (.*)")

In [72]:
data = open("data.txt", "r").read()

In [73]:
matches = pattern.findall(data)

In [74]:
from dataclasses import dataclass
@dataclass
class Valve:
    name: str
    flow_rate: int
    edges: list[str]

In [75]:
valve_map: dict[str, Valve] = {}

In [76]:
for match in matches:
    valve_map[match[0]] = Valve(match[0], int(match[1]), match[2].split(", "))

In [77]:
valve_map

{'AW': Valve(name='AW', flow_rate=0, edges=['LG', 'TL']),
 'OM': Valve(name='OM', flow_rate=0, edges=['XK', 'IM']),
 'BG': Valve(name='BG', flow_rate=0, edges=['MP', 'SB']),
 'XB': Valve(name='XB', flow_rate=0, edges=['MA', 'TL']),
 'CD': Valve(name='CD', flow_rate=0, edges=['VL', 'OF']),
 'VF': Valve(name='VF', flow_rate=0, edges=['CS', 'XK']),
 'HK': Valve(name='HK', flow_rate=0, edges=['RL', 'QB']),
 'QN': Valve(name='QN', flow_rate=0, edges=['IV', 'QR']),
 'OF': Valve(name='OF', flow_rate=4, edges=['TQ', 'CD', 'IR', 'IM', 'JE']),
 'QB': Valve(name='QB', flow_rate=14, edges=['HK', 'XE', 'CS', 'VO']),
 'ZE': Valve(name='ZE', flow_rate=7, edges=['JB', 'NC', 'SE', 'OI']),
 'OW': Valve(name='OW', flow_rate=0, edges=['MB', 'JB']),
 'MA': Valve(name='MA', flow_rate=0, edges=['XB', 'MB']),
 'MP': Valve(name='MP', flow_rate=0, edges=['VK', 'BG']),
 'UE': Valve(name='UE', flow_rate=9, edges=['ZM', 'RZ', 'WI', 'HO', 'FO']),
 'QR': Valve(name='QR', flow_rate=24, edges=['QN']),
 'TQ': Valve(nam

## Precompute paths (BFS)

In [78]:
def generate_path(from_valve: str, to_valve: str):
    stack: list[tuple[str]] = [(from_valve,)]
    visited = set()
    while len(stack) > 0:
        hand = stack.pop(0)

        if hand[-1] == to_valve:
            return hand

        for edge in valve_map[hand[-1]].edges:
            if edge in visited:
                continue
            stack.append(hand + (edge,))
            visited.add(edge)

In [79]:
no_flow = []
for valve in valve_map.keys():
    if valve_map[valve].flow_rate == 0:
        no_flow.append(valve)

In [80]:
path_map: dict[str, dict[str, tuple[str]]] = {}
for a in valve_map.keys():
    if a in no_flow and a != "AA":
        continue
    path_map[a] = {}
    for b in valve_map.keys():
        if b in no_flow:
            continue
        path_map[a][b] = generate_path(a,b)[1:]

## DFS (by path, full search)

In [81]:
def find_max_released(max_time=30):
    max_released = 0
    # Stack of (current_position, opened_valves, released, time_passed)
    stack: list[tuple[str, tuple[str], int, int]] = [("AA", tuple(no_flow), 0, 1)]
    while len(stack):
        # Pop
        current_position, opened_valves, released, time_passed = stack.pop()
        # Process
        max_released = max(released, max_released)
        # Push (want to push whichever is lower
        if len(opened_valves) == len(valve_map) or time_passed >= max_time:
            continue
        for target in path_map[current_position].keys():
            if target in opened_valves:
                continue
            end_time = min(time_passed + len(path_map[current_position][target]), max_time)
            next_released = released + ((max_time - end_time) * valve_map[target].flow_rate if end_time <= max_time else 0)
            stack.append((
                target,
                opened_valves + (target,),
                next_released,
                end_time + 1
            ))
    return max_released

In [82]:
print(find_max_released(30))

1789


# Part 2
## DFS (by path, full search)

In [83]:
def find_max_released_elephant(max_time=30):
    max_released = 0
    # Stack of (current_position, elephant_position, opened_valves, released, time_passed, elephant_passed)
    stack: list[tuple[str, str, tuple[str], int, int, int]] = [("AA", "AA", tuple(no_flow), 0, 1, 1)]
    while len(stack):
        # Pop
        current_position, elephant_position, opened_valves, released, time_passed, elephant_passed = stack.pop()
        # Process
        max_released = max(released, max_released)
        # Push
        lowest_time = min(time_passed, elephant_passed)
        choice = "you" if lowest_time == time_passed else "elephant"
        if len(opened_valves) == len(valve_map) or lowest_time >= max_time:
            continue
        for target in path_map[current_position if choice == "you" else elephant_position].keys():
            if target in opened_valves:
                continue
            end_time = min((time_passed if choice == "you" else elephant_passed) +
                           len(path_map[current_position if choice == "you" else elephant_position][target]), max_time)
            next_released = released + ((max_time - end_time) * valve_map[target].flow_rate if end_time <= max_time else 0)
            stack.append((
                target if choice == "you" else current_position,
                target if choice == "elephant" else elephant_position,
                opened_valves + (target,),
                next_released,
                end_time + 1 if choice == "you" else time_passed,
                end_time + 1 if choice == "elephant" else elephant_passed,
            ))
    return max_released

In [84]:
print(find_max_released_elephant(26))

2496


### Very SLOW!